In [6]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
import numpy as np
import pandas as pd
password = getpass.getpass()

In [7]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [32]:
# he creado una query en Mysql y no me ha funcionado en python. importo una query mas basica. (SOLUCIONADO HAY QUE AGREGAR DOS % %)

#query = '''SELECT F.film_id, F.title, date_format(convert(R.Rental_date, date), '%%M') as rental_month_number, date_format(convert(R.Rental_date, date), '%%Y') as rental_year FROM FILM F JOIN inventory I USING (film_id) JOIN RENTAL R USING (inventory_id) GROUP BY F.film_id, F.title,rental_month_number, rental_year;'''

#data = pd.read_sql_query(query, engine) # engine se puede cambiar por la URL connection_string
#data.head()

,film_id,title,rental_month_number,rental_year
0,1,ACADEMY DINOSAUR,July,2005
1,1,ACADEMY DINOSAUR,August,2005
2,1,ACADEMY DINOSAUR,May,2005
3,1,ACADEMY DINOSAUR,June,2005
4,1,ACADEMY DINOSAUR,May,2023


In [84]:

query = """SELECT F.title, c.name AS category, f.rental_duration, f.rental_rate, f.length, r.rental_date
FROM film f
JOIN inventory i USING (film_id)
JOIN rental r USING (inventory_id)
JOIN film_category fc USING (film_id)
JOIN category c USING (category_id)
GROUP BY F.film_id, c.name, f.rental_duration, f.rental_rate, f.length, r.rental_date;;"""
data = pd.read_sql_query(query, engine) # engine se puede cambiar por la URL connection_string
data

,title,category,rental_duration,rental_rate,length,rental_date
0,AMADEUS HOLY,Action,6,0.99,113,2005-08-02 01:16:59
1,AMADEUS HOLY,Action,6,0.99,113,2005-08-18 04:26:54
2,AMADEUS HOLY,Action,6,0.99,113,2005-06-20 20:35:28
3,AMADEUS HOLY,Action,6,0.99,113,2005-07-09 05:01:58
4,AMADEUS HOLY,Action,6,0.99,113,2005-07-30 08:02:39
...,...,...,...,...,...,...
16026,WORKING MICROCOSMOS,Travel,4,4.99,74,2005-05-30 18:45:17
16027,WORKING MICROCOSMOS,Travel,4,4.99,74,2005-06-15 09:03:52
16028,WORKING MICROCOSMOS,Travel,4,4.99,74,2005-07-07 06:38:31
16029,WORKING MICROCOSMOS,Travel,4,4.99,74,2005-07-30 00:12:41


In [43]:
# Convertir la columna "rental_date" a tipo datetime
data['rental_date'] = pd.to_datetime(data['rental_date'])

# Crear una nueva columna para el mes
data['rental_month'] = data['rental_date'].dt.month

# Crear una nueva columna para el año
data['rental_year'] = data['rental_date'].dt.year

# Mostrar el resultado
print(data[['rental_month', 'rental_year']])

       rental_month  rental_year
0                 8         2005
1                 8         2005
2                 6         2005
3                 7         2005
4                 7         2005
...             ...          ...
16026             5         2005
16027             6         2005
16028             7         2005
16029             7         2005
16030             8         2005

[16031 rows x 2 columns]


In [50]:
# voy a eliminar la columna rental_date, no me sirve ese formato 
data2 = data.drop('rental_date', axis=1)
data2.head()

,title,category,rental_duration,rental_rate,length,rental_month,rental_year
0,AMADEUS HOLY,Action,6,0.99,113,8,2005
1,AMADEUS HOLY,Action,6,0.99,113,8,2005
2,AMADEUS HOLY,Action,6,0.99,113,6,2005
3,AMADEUS HOLY,Action,6,0.99,113,7,2005
4,AMADEUS HOLY,Action,6,0.99,113,7,2005


In [51]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16031 entries, 0 to 16030
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            16031 non-null  object 
 1   category         16031 non-null  object 
 2   rental_duration  16031 non-null  int64  
 3   rental_rate      16031 non-null  float64
 4   length           16031 non-null  int64  
 5   rental_month     16031 non-null  int64  
 6   rental_year      16031 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 876.8+ KB


In [49]:
# COPIA DE SEGURIDAD
data3=data2.copy()

In [54]:
# trabajo con variables numericas. 
data_numeric = data3[['rental_duration','rental_rate','length','rental_month','rental_year']]
data_numeric.head()

,rental_duration,rental_rate,length,rental_month,rental_year
0,6,0.99,113,8,2005
1,6,0.99,113,8,2005
2,6,0.99,113,6,2005
3,6,0.99,113,7,2005
4,6,0.99,113,7,2005


In [55]:
#Pre- procesamiento. Normalizo variables numericas. 
X = data_numeric
transformer = Normalizer().fit(X)
x_normalized = transformer.transform(X)
data_norm=pd.DataFrame(x_normalized)

In [64]:
col_num=data_numeric.columns
data_norm.columns=col_num
data_norm

,rental_duration,rental_rate,length,rental_month,rental_year
0,0.002988,0.000493,0.056269,0.003984,0.998403
1,0.002988,0.000493,0.056269,0.003984,0.998403
2,0.002988,0.000493,0.056269,0.002988,0.998407
3,0.002988,0.000493,0.056269,0.003486,0.998405
4,0.002988,0.000493,0.056269,0.003486,0.998405
...,...,...,...,...,...
16026,0.001994,0.002487,0.036882,0.002492,0.999311
16027,0.001994,0.002487,0.036882,0.002990,0.999310
16028,0.001994,0.002487,0.036882,0.003489,0.999308
16029,0.001994,0.002487,0.036882,0.003489,0.999308


In [58]:
# TRABAJO CON VARIABLES CATEGORICAS 
data_categ= data3[['title','category']]

In [63]:
# se viene la dummisación de variables. 
data_dummies = pd.get_dummies(data_categ)
data_dummies

,title_ACADEMY DINOSAUR,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,...,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16029,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [65]:
# creo un nuevo dataframe con una variable booleana de verdadero o falso 
data_variable_dependiente = data3[['rental_month','rental_year']]
data_variable_dependiente

,rental_month,rental_year
0,8,2005
1,8,2005
2,6,2005
3,7,2005
4,7,2005
...,...,...
16026,5,2005
16027,6,2005
16028,7,2005
16029,7,2005


In [67]:
# creo la nueva columna 
data_variable_dependiente['target_variable'] = ((data_variable_dependiente['rental_month'] == 7) & (data_variable_dependiente['rental_year'] == 2005)).astype(int)
data_variable_dependiente

C:\Users\roriv\AppData\Local\Temp\ipykernel_29096\3279901775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_variable_dependiente['target_variable'] = ((data_variable_dependiente['rental_month'] == 7) & (data_variable_dependiente['rental_year'] == 2005)).astype(int)


,rental_month,rental_year,target_variable
0,8,2005,0
1,8,2005,0
2,6,2005,0
3,7,2005,1
4,7,2005,1
...,...,...,...
16026,5,2005,0
16027,6,2005,0
16028,7,2005,1
16029,7,2005,1


In [72]:
# quito las columnas de comparación mes y año, me quedo solo con el verdadero falso. 
df_target_variable = data_variable_dependiente.drop(['rental_month', 'rental_year'], axis=1)
df_target_variable

,target_variable
0,0
1,0
2,0
3,1
4,1
...,...
16026,0
16027,0
16028,1
16029,1


In [74]:
# preparo el DF para mi modelo, concateno df numerico normalizado, df categorico con dummies y la nueva variable objetivo. 
data_model = pd.concat([df_target_variable, data_dummies, data_norm], axis=1)
data_model

,target_variable,title_ACADEMY DINOSAUR,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,...,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel,rental_duration,rental_rate,length,rental_month,rental_year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.002988,0.000493,0.056269,0.003984,0.998403
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.002988,0.000493,0.056269,0.003984,0.998403
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.002988,0.000493,0.056269,0.002988,0.998407
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.002988,0.000493,0.056269,0.003486,0.998405
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.002988,0.000493,0.056269,0.003486,0.998405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.001994,0.002487,0.036882,0.002492,0.999311
16027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.001994,0.002487,0.036882,0.002990,0.999310
16028,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.001994,0.002487,0.036882,0.003489,0.999308
16029,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.001994,0.002487,0.036882,0.003489,0.999308


In [78]:
# divido las variables 
Y = data_model['target_variable']
X = data_model.drop('target_variable', axis=1)

In [81]:
#aplico el Train test
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=100)


In [82]:
# aplico modelo de regresión logisticca.
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train, Y_train)

In [83]:
predictions = classification.predict(X_test)
classification.score(X_test, Y_test)

0.5105254950881023

In [86]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[2922,  823],
       [2316,  352]], dtype=int64)

In [85]:
classification.predict_proba(X_test)

array([[0.58852062, 0.41147938],
       [0.48105382, 0.51894618],
       [0.61355991, 0.38644009],
       ...,
       [0.46703342, 0.53296658],
       [0.70247274, 0.29752726],
       [0.31524214, 0.68475786]])

In [97]:
# Make predictions for complete dataframe

all_predictions = classification.predict(X)
all_predictions

# Create a new column 'predicted_rented' in the concatenated DataFrame

df_predictions = data3 # df_lr is DataFrame with all variables used in the logistic regression model.
df_predictions['predicted_rented'] = all_predictions # Insert predicted_rented column results to DataFrame
df_predictions

# Create DataFrame with relevant variables to show results

films_predictions = df_predictions[[ 'title', 'predicted_rented']]
films_predictions.sample(30)


,title,predicted_rented
15668,LOCK REAR,0
8572,GENTLEMEN STAGE,0
5462,FRENCH HOLIDAY,0
6882,SCORPION APOLLO,0
10971,SPIRIT FLINTSTONES,1
13668,MOURNING PURPLE,0
12200,DAY UNFAITHFUL,0
9424,CURTAIN VIDEOTAPE,0
2811,MAGIC MALLRATS,1
1673,INTENTIONS EMPIRE,1


### Evaluación de resultados. 

Al realizar la regresión, obtengo un modelo con un resultado insatisfactorio. Analizando la variable dependiente, veo que está muy desbalanceada. Seria optimo trabajar ese desalance, acotando DF solo quedandome con las variables que cumplan la condición de que han sido alquiladas el mes anterior. 